In [36]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<main>
<article class="day-desc read-aloud"><h2>--- Day 8: Memory Maneuver ---</h2><p>The sleigh is much easier to pull than you'd expect for something its weight. Unfortunately, neither you nor the Elves know <span title="It's North. Obviously.">which way</span> the North Pole is from here.</p>
<p>You check your wrist device for anything that might help.  It seems to have some kind of navigation system!  Activating the navigation system produces more bad news: "Failed to start navigation system. Could not read software license file."</p>
<p>The navigation system's license file consists of a list of numbers (your puzzle input).  The numbers define a data structure which, when processed, produces some kind of <a href="https://en.wikipedia.org/wiki/Tree_(data_structure)">tree</a> that can be used to calculate the license number.</p>
<p>The <em>tree</em> is made up of <em>nodes</em>; a single, outermost node forms the tree's <em>root</em>, and it contains all other nodes in the tree (or contains nodes that contain nodes, and so on).</p>
<p>Specifically, a node consists of:</p>
<ul>
<li>A <em>header</em>, which is always exactly two numbers:
<ul>
<li>The quantity of child nodes.</li>
<li>The quantity of metadata entries.</li>
</ul>
</li><li>Zero or more <em>child nodes</em> (as specified in the header).</li>
<li>One or more <em>metadata entries</em> (as specified in the header).</li>
</ul>
<p>Each child node is itself a node that has its own header, child nodes, and metadata. For example:</p>
<pre><code>2 3 0 3 10 11 12 1 1 0 1 99 2 1 1 2
A----------------------------------
    B----------- C-----------
                     D-----
</code></pre>
<p>In this example, each node of the tree is also marked with an underline starting with a letter for easier identification. In it, there are four nodes:</p>
<ul>
<li><code>A</code>, which has <code>2</code> child nodes (<code>B</code>, <code>C</code>) and <code>3</code> metadata entries (<code>1</code>, <code>1</code>, <code>2</code>).</li>
<li><code>B</code>, which has <code>0</code> child nodes and <code>3</code> metadata entries (<code>10</code>, <code>11</code>, <code>12</code>).</li>
<li><code>C</code>, which has <code>1</code> child node (<code>D</code>) and <code>1</code> metadata entry (<code>2</code>).</li>
<li><code>D</code>, which has <code>0</code> child nodes and <code>1</code> metadata entry (<code>99</code>).</li>
</ul>
<p>The first check done on the license file is to simply add up all of the metadata entries.  In this example, that sum is <code>1+1+2+10+11+12+2+99=<em>138</em></code>.</p>
<p><em>What is the sum of all metadata entries?</em></p>
</article>

</main>


In [37]:
from more_itertools import first


example = "2 3 0 3 10 11 12 1 1 0 1 99 2 1 1 2"


def sum_metadata(license: str) -> int:
    def process_node(i: int) -> tuple[int, int]:
        childs = nodes[i]
        meta = nodes[i + 1]

        i += 2
        total_sum_meta = 0
        for _ in range(childs):
            meta_sum, i = process_node(i)
            total_sum_meta += meta_sum

        total_sum_meta += sum(nodes[j] for j in range(i, i + meta))
        return total_sum_meta, i + meta

    nodes = [int(i) for i in license.split()]

    return first(process_node(0))


print(f"Example: {sum_metadata(example)} should be 1+1+2+10+11+12+2+99=138")

Example: 138 should be 1+1+2+10+11+12+2+99=138


In [38]:
with open("../input/day8.txt") as f:
    puzzle = f.read().strip()

print(f"Part I: {sum_metadata(puzzle)}")

Part I: 42501


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>42501</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>The second check is slightly more complicated: you need to find the value of the root node (<code>A</code> in the example above).</p>
<p>The <em>value of a node</em> depends on whether it has child nodes.</p>
<p>If a node has <em>no child nodes</em>, its value is the sum of its metadata entries. So, the value of node <code>B</code> is <code>10+11+12=33</code>, and the value of node <code>D</code> is <code>99</code>.</p>
<p>However, if a node <em>does have child nodes</em>, the metadata entries become indexes which refer to those child nodes. A metadata entry of <code>1</code> refers to the first child node, <code>2</code> to the second, <code>3</code> to the third, and so on. The value of this node is the sum of the values of the child nodes referenced by the metadata entries. If a referenced child node does not exist, that reference is skipped. A child node can be referenced multiple time and counts each time it is referenced. A metadata entry of <code>0</code> does not refer to any child node.</p>
<p>For example, again using the above nodes:</p>
<ul>
<li>Node <code>C</code> has one metadata entry, <code>2</code>. Because node <code>C</code> has only one child node, <code>2</code> references a child node which does not exist, and so the value of node <code>C</code> is <code>0</code>.</li>
<li>Node <code>A</code> has three metadata entries: <code>1</code>, <code>1</code>, and <code>2</code>. The <code>1</code> references node <code>A</code>'s first child node, <code>B</code>, and the <code>2</code> references node <code>A</code>'s second child node, <code>C</code>. Because node B has a value of <code>33</code> and node <code>C</code> has a value of <code>0</code>, the value of node <code>A</code> is <code>33+33+0=<em>66</em></code>.</li>
</ul>
<p>So, in this example, the value of the root node is <code>66</code>.</p>
<p><em>What is the value of the root node?</em></p>
</article>

</main>


In [39]:
from collections import Counter


def value_of_root_node(license: str) -> int:
    def process_node(i: int) -> tuple[int, int]:
        childs = nodes[i]
        meta = nodes[i + 1]
        i += 2

        if childs == 0:
            return sum(nodes[i : i + meta]), i + meta

        child_values = []
        for _ in range(childs):
            value, i = process_node(i)
            child_values.append(value)

        meta_counts = Counter(nodes[i : i + meta])
        value = sum(meta_counts[i] * v for i, v in enumerate(child_values, start=1))
        return value, i + meta

    nodes = [int(i) for i in license.split()]

    return first(process_node(0))


print(f"Example: {value_of_root_node(example)} should be 33+33+0=66")

Example: 66 should be 33+33+0=66


In [40]:
print(f"Part I: {value_of_root_node(puzzle)}")

Part I: 30857


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>30857</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
